# Exploring the Data

> Initial exploration of the data

In [ ]:
#| default_exp explore_data

In [ ]:
#| export
import kaggle
from pathlib import Path

In [ ]:
#| export
download_path = Path("../../tmp")
download_path.mkdir(exist_ok=True)
dataset_suffix = "ruchi798/data-science-job-salaries"

In [ ]:
kaggle.api.dataset_download_files(dataset=dataset_suffix, path=download_path, unzip=True)

In [ ]:
!ls ../../tmp

ds_salaries.csv


In [ ]:
import pandas as pd

In [ ]:
#| export
def download_data():
    kaggle.api.dataset_download_files(dataset=dataset_suffix, path=download_path, unzip=True)
    return pd.read_csv(download_path / "ds_salaries.csv", index_col=0).head()

In [ ]:
data = download_data()
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
